This Project was completed using OpenCv only, If you hope to find Deep Learning and Machine Learning in this project then you won't find any!!!

This Project was done using 6 steps:

    
1) Find the color range of the target object and save it

2) Apply the correct morphological operation to reduce noise in the video

3) Detect and track the colored object with contour detection.

4) Find the object's x,y location coordinates to draw on the screen

5) Add a Wiper Functionality to wipe off the Whole screen

6) Add an Eraser Functionality to erase parts of the drawing

Before moving on I strongly recommend you to check Edge detection, Countor Detection, BacgroundSubtractor and look how to find them in OpenCV and all the Parameteres they accept

And do check about RBG, HSV

Ok lets get Started!!!